# LOS Tabular

## Setup

In [9]:
# First install package from terminal:
!pip install -U pip
!pip install -U setuptools wheel
!pip install autogluon  # autogluon==0.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
# import all required modules
import pandas as pd
import numpy as np
import os
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)  ###
pd.set_option('display.max_columns', None)  ###
pd.set_option('display.width', None)  ###
pd.set_option('display.max_colwidth', None)  ###

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.metrics import cohen_kappa_score
from sklearn.inspection import permutation_importance
from autogluon.tabular import TabularPredictor

In [11]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:  
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/MyDrive/MIMIC-III Text Mining/LOS_FINAL/"

else:
  # Setup Repository
  with open("repo_info.txt", "r") as repo_info:
      path_to_repo = repo_info.readline()

  
print(path_to_repo)

path_to_data = f"{path_to_repo}data/"
path_to_raw = f"{path_to_data}raw/"
path_to_processed = f"{path_to_data}processed/"
path_to_lda = f"{path_to_data}lda/"
path_to_icd = f"{path_to_data}icd_codes/"
path_to_models = f"{path_to_repo}models/"
path_to_results = f"{path_to_repo}results/"

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/MIMIC-III Text Mining/LOS_FINAL/


## Import the dataset

In [12]:
def load_datasets(method):
    """
    Function to load train, test and validation set based on the chosen method
    method: string for the processing method we want to load
    """
    global path_to_processed
    # load it back
    train = pd.read_feather('{}{}train_{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag))
    test = pd.read_feather('{}{}test_{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag))
    return train, test

In [13]:
# PARAMETERS

session_seed = 42 # set seed for our session
seed_tag = f'_{session_seed}'

random.seed(session_seed)

## Train the Tabular Model (Structured DataSet)

In [14]:
# Model Parameters
save_path = f'{path_to_models}tabular'
label = 'los_cat'
metric = 'roc_auc'

In [15]:
# compute other metrics
def perf_evaluator(y_test, y_pred, y_pred_proba):
    """ Function to display the main classification performance metrics """
    kappa = cohen_kappa_score(y_test, y_pred)
    precision, recall, prc_th = precision_recall_curve(y_test, y_pred_proba)
    prc_auc = auc(recall, precision)
    return kappa, prc_auc

In [16]:
# Iterate over our main methods of vectorization

vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}

for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_{method}')
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)

        # evaluate performance on the test set
        per_tab = predictor.evaluate(test)
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['los_cat'], y_pred, y_pred_proba)
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_{method}.xlsx')
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_{method}.xlsx')
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_{method}.xlsx')

tabular
frequency


Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC-III Text Mining/LOS_FINAL/models/tabular/"
AutoGluon Version:  0.6.0
Python Version:     3.7.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    24611
Train Data Columns: 46
Label Column: los_cat
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [False, True]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = True, class 0 = False
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    52812.1 MB
	Train Data (Original)  Memory Usage: 28.97 MB (0.1% o


Test set performance:
{'roc_auc': 0.9441635626643294, 'accuracy': 0.9471802372826263, 'balanced_accuracy': 0.7051531003505697, 'mcc': 0.5348751697099995, 'f1': 0.5376955903271693, 'precision': 0.7411764705882353, 'recall': 0.421875}

Performance metrics:
{"Cohen's Kappa": 0.5119144639388208, 'PRC AUC': 0.6552421524932525}


Computing feature importance via permutation shuffling for 46 features using 5000 rows with 5 shuffle sets...


                  model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.944164   0.947699        1.041939       0.656059  73.975200                 0.007094                0.000753           0.783005            2       True         14
1            LightGBMXT    0.942363   0.941051        0.058727       0.016287   1.245087                 0.058727                0.016287           1.245087            1       True          3
2              CatBoost    0.942151   0.942311        0.027428       0.015730  10.103743                 0.027428                0.015730          10.103743            1       True          7
3              LightGBM    0.939862   0.943836        0.029292       0.057942   1.338549                 0.029292                0.057942           1.338549            1       True          4
4         LightGBMLarge    0.939767   0.

...


Test set performance:
{'roc_auc': 0.9531876017278077, 'accuracy': 0.9488054607508533, 'balanced_accuracy': 0.7173422436459247, 'mcc': 0.5543387184765703, 'f1': 0.5594405594405595, 'precision': 0.7490636704119851, 'recall': 0.44642857142857145}

Performance metrics:
{"Cohen's Kappa": 0.534105995366047, 'PRC AUC': 0.6828141267633003}


Computing feature importance via permutation shuffling for 346 features using 5000 rows with 5 shuffle sets...


                  model  score_test  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.953188   0.957610        1.946153       1.098867  133.877510                 0.006784                0.000717           0.670780            2       True         14
1              CatBoost    0.951734   0.951627        0.046385       0.032380   26.638586                 0.046385                0.032380          26.638586            1       True          7
2            LightGBMXT    0.949801   0.949965        0.101268       0.025269    6.258527                 0.101268                0.025269           6.258527            1       True          3
3              LightGBM    0.948555   0.952760        0.045216       0.021150    6.107953                 0.045216                0.021150           6.107953            1       True          4
4        NeuralNetTorch    0.947751

	3036.56s	= Expected runtime (607.31s per shuffle set)
	1604.95s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                      importance    stddev   p_value  n  p99_high   p99_low  \
urea_n_max          2.692636e-02  0.002261  0.000006  5  0.031581  0.022271   
platelets_min       2.049098e-02  0.003156  0.000065  5  0.026989  0.013993   
platelets_max       1.808519e-02  0.003357  0.000136  5  0.024997  0.011174   
urea_n_min          1.385697e-02  0.001052  0.000004  5  0.016023  0.011690   
F205                9.817731e-03  0.002183  0.000275  5  0.014312  0.005324   
type_stay           4.665051e-03  0.001606  0.001450  5  0.007972  0.001358   
platelets_mean      3.820603e-03  0.001213  0.001073  5  0.006319  0.001322   
temp_max            3.201463e-03  0.001079  0.001339  5  0.005423  0.000980   
F123                2.379211e-03  0.000292  0.000027  5  0.002980  0.001779   
F122                2.130760e-03  0.000284  0.000037  5  0.002715  0.001547   
F16                 1.982486e-03  0.000583  0.000804  5  0.003183  0.000782   
F185                1.827332e-0